# 1. SETTINGS

In [ ]:
# import packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
import scipy.stats
import os
import time

In [ ]:
# helper functions
import functions
from functions import *

In [ ]:
# pandas options
pd.set_option('display.max_columns', None)

In [ ]:
# dark background style
plt.style.use('dark_background')

In [ ]:
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# garbage collection
import gc
gc.enable()

# 2. DATA PREPARATION

In [ ]:
### extract test IDs

# import CSV
df = pd.read_csv('../data/data_v3.csv')

# partitioning
target = 'fraud'
test  = df[df[target].isnull() == True]

# extract IDs
test_ids = test['id']
del test

In [ ]:
### IMPORT PREDS

# list names
names = sorted(os.listdir('../submissions'))

# preprocessing loop
for name in names:

    # load preds
    tmp_te = pd.read_csv('../submissions/' + str(name))
    tmp_te.columns = [name]    

    # cbind data
    if name == names[0]:     
        test  = tmp_te
    else:
        test  = pd.concat([test,  tmp_te], axis = 1)
        
# display information
print('- Data shape:',  test.shape)

# 3. BLENDING

In [ ]:
# select models
features = [f for f in test.columns if 'stacking_seed' in f]
print(test[features].shape)

In [ ]:
# mean ensemble
means = np.array(test[features].mean(axis = 1))

# 4. SUBMISSION

In [ ]:
# file name
name = 'mean_ensemble'

In [ ]:
# check submission
sub = pd.DataFrame({'id': test_ids, 'fraud': means})
sub['fraud'] = np.round(sub['fraud']).astype('int')
sub.head()

In [ ]:
# export submission
sub = sub[['fraud']]
sub.to_csv('../submissions/' + str(name) + '.csv', index = False)
sub.shape

In [ ]:
# check correlation with best individual submission
prev_sub = pd.read_csv('../submissions/lgb_v8_375.csv')
cor = np.sum(prev_sub[target] == sub.reset_index()[target]) / len(sub)
print("Share of the same predictions: " + str(np.round(cor, 6)))